In [ ]:
#|default_exp mic

# Mic streaming and recording

In [ ]:
#| export
import sys
import zmq
import sounddevice as sd

In [ ]:
#| export 
from asr_teach.utils import ZMQ_ARGS, PROTOCOL, ADDR, PORT

In [ ]:
#| export

class MicStream:
    def __init__(self,
                 sample_rate: float = None,
                 device: int = 0,
                 block_duration: int = 50,
                 addr: str = ADDR,
                 port: int = PORT,
                 protocol: str = PROTOCOL):
        
        # create the socket
        context = context or zmq.Context()
        socket = context.socket(zmq.PUSH)
        socket.bind(f'{protocol}://{addr}:{port}')
        self.socket = socket

        # set the sample rate
        sample_rate = sample_rate or  sd.query_devices(device, 'input')['default_samplerate']

        def callback(indata, frames, time, status):
            '''Sends a numpy array over a zmq port.
            '''
            if any(indata):
                # sends the first message with array shape and data type
                md = {'dtype': str(indata.dtype),
                      'shape': indata.shape}
                socket.send_json(md, zmq.SNDMORE)
                # sends the array data
                socket.send(indata, flags, copy=copy, track=track)
        
        # create the microphone streaming object
        self.stream = sd.InputStream(device=device,
                                     channels=1,
                                     callback=callback,
                                     blocksize=int(sample_rate * block_duration / 1000),
                                     samplerate=sample_rate)
        self.stream = stream
        
    def run(self):
        '''Streams audio until the user stops or interrupts the process.
        '''
        try:
            # start streaming with context manager
            with self.stream:
                while True:
                    
                    # monitor for user interrupts
                    response = input()
                    if response in ('', 'q', 'Q'):
                        break

        except KeyboardInterrupt:
            print('User set keyboard interrupt.')
            sys.exit(0)
        except Exception as e:
            print(f'Exception: {e}')
            raise
